In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [3]:
target = train.TARGET.values.copy()
train.drop('TARGET', axis=1, inplace=True)

In [4]:
train.loc[:, 'n0'] = (train == 0).sum(axis=1)
test.loc[:, 'n0'] = (test == 0).sum(axis=1)

In [5]:
dlist = list()
for c in train.columns:
    if train[c].unique().shape[0] == 1:
        print(c)
        dlist.append(c)
train.drop(dlist, axis=1, inplace=True)
test.drop(dlist, axis=1, inplace=True)

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var33_hace3
imp_trasp_var17_out_hace3
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var33_hace3
num_trasp_var17_out_hace3
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3


In [6]:
for cone in train.columns:
    for ctwo in train.columns:
        if cone not in train.columns:
            continue
        if ctwo not in train.columns:
            continue
        if cone != ctwo and np.all(train[cone] == train[ctwo]):
            print(cone, ctwo)
            train.drop(ctwo, axis=1, inplace=True)
            test.drop(ctwo, axis=1, inplace=True)

ind_var6_0 ind_var29_0
ind_var6 ind_var29
ind_var13_medio_0 ind_var13_medio
ind_var18_0 ind_var18
ind_var26_0 ind_var26
ind_var25_0 ind_var25
ind_var32_0 ind_var32
ind_var34_0 ind_var34
ind_var37_0 ind_var37
ind_var40 ind_var39
num_var6_0 num_var29_0
num_var6 num_var29
num_var13_medio_0 num_var13_medio
num_var18_0 num_var18
num_var26_0 num_var26
num_var25_0 num_var25
num_var32_0 num_var32
num_var34_0 num_var34
num_var37_0 num_var37
num_var40 num_var39
saldo_var6 saldo_var29
saldo_var13_medio saldo_medio_var13_medio_ult1
delta_imp_reemb_var13_1y3 delta_num_reemb_var13_1y3
delta_imp_reemb_var17_1y3 delta_num_reemb_var17_1y3
delta_imp_reemb_var33_1y3 delta_num_reemb_var33_1y3
delta_imp_trasp_var17_in_1y3 delta_num_trasp_var17_in_1y3
delta_imp_trasp_var17_out_1y3 delta_num_trasp_var17_out_1y3
delta_imp_trasp_var33_in_1y3 delta_num_trasp_var33_in_1y3
delta_imp_trasp_var33_out_1y3 delta_num_trasp_var33_out_1y3


In [7]:
for c in train.columns:
    cmin = train[c].min()
    cmax = train[c].max()
    test.loc[test[c] < cmin, c] = cmin
    test.loc[test[c] > cmax, c] = cmax

In [8]:
import xgboost as xgb

In [12]:
train.loc[:, 'var3'] += 1

In [19]:
train.loc[train['var3'] == -999998, 'var3'] = 0

In [20]:
dtrain = xgb.DMatrix(train, target, missing=0)

In [21]:
params = {'objective': 'binary:logistic', 
          'eval_metric': 'auc',
          'eta': 0.0202048,
          'max_depth': 5,
          'subsample': 0.6815,
          'colsample_bytree': 0.701,
          'silent': 1,
          'seed': 0
}

In [22]:
xgb.cv(params, dtrain, num_boost_round=560, nfold=5, stratified=True, verbose_eval=True)

[0]	cv-test-auc:0.7700326+0.048758743645012	cv-train-auc:0.7812764+0.03670268264636795
[1]	cv-test-auc:0.7842422+0.04106154800004503	cv-train-auc:0.7985659999999999+0.03079161490406116
[2]	cv-test-auc:0.8035676+0.016940033466318765	cv-train-auc:0.8165017999999999+0.011725178129137323
[3]	cv-test-auc:0.8055362+0.01935373581921589	cv-train-auc:0.8192692000000001+0.01284145800756287
[4]	cv-test-auc:0.8058254+0.019672278765816623	cv-train-auc:0.8206178000000002+0.01300484264264664
[5]	cv-test-auc:0.8086926+0.01599238425751458	cv-train-auc:0.823179+0.008381742945235202
[6]	cv-test-auc:0.8097863999999999+0.01739960831283282	cv-train-auc:0.8236437999999999+0.008448965721317607
[7]	cv-test-auc:0.8113923999999999+0.01732308827663242	cv-train-auc:0.8250321999999999+0.007859628016643035
[8]	cv-test-auc:0.8126672000000001+0.016101971089279736	cv-train-auc:0.827+0.006550131326927715
[9]	cv-test-auc:0.8135304+0.014931367232775432	cv-train-auc:0.8286122+0.0063762966806760035
[10]	cv-test-auc:0.814586

KeyboardInterrupt: 

In [25]:
test.loc[:, 'var3'] += 1
test.loc[test['var3'] == -999998, 'var3'] = 0

In [26]:
dtest = xgb.DMatrix(test, missing=0)

In [27]:
gbm = xgb.train(params, dtrain, num_boost_round=560)

In [28]:
preds = gbm.predict(dtest)

In [29]:
preds

array([ 0.04661046,  0.05412474,  0.00133519, ...,  0.00337909,
        0.06087907,  0.00106202], dtype=float32)

In [30]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = preds
sample.to_csv('submission/init_xgb_0v3.csv', index_label='ID')